In [2]:
#Clickhouse db w/ Goldsky

import requests as r
import pandas as pd
import clickhouse_connect as cc


import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
import clickhouse_utils as ch
sys.path.pop()

import time

client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# client.close()

In [ ]:
chain_mappings = {
        'zora': 'Zora',
        'pgn': 'Public Goods Network',
        'base': 'Base',
        'mode': 'Mode',
        'metal': 'Metal',
        'frax': 'Fraxtal'
        # Add more mappings as needed
    }

In [ ]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "frax_allltime_chain_activity"
]

In [ ]:
#checks
# print(pandas_gbq.context.project)
# print(pandas_gbq.context.credentials)
dune_df = []
dune_table = 'op_gcs_allltime_chain_activity'

In [ ]:
for qn in query_names:
        # If we can do it programmatically from UI saved queries
        # query = client.get_job(query_name)
        # Read the SQL query from file
        with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                query = file.read()
        print(qn)
        table_name = qn
        
        # Execute the query
        query_job = client.query(query)
        # df = pandas_gbq.read_gbq(query)
        # Fetch the results
        results = query_job.result()
        # Convert the results to a DataFrame
        df = results.to_dataframe()
        
        # Write to csv
        df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        # print(df.sample(5))
        time.sleep(1)
        
        # Write to Dune
        df['chain_raw'] = df['chain']
        df['chain'] = df['chain'].replace(chain_mappings)
        dune_df.append(df)

write_df = pd.concat(dune_df)
d.write_dune_api_from_pandas(write_df, dune_table,table_description = dune_table)
write_df.to_csv('outputs/chain_data/' + dune_table + '.csv', index=False)
# Print the results

In [ ]:
print(write_df.sort_values(by='dt',ascending=False))